In [38]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re
import urllib

In [39]:
#连接 / 光标模式是数据库编程中常用的模式
#连接模式除了要连接数据库之外，还要发送数据库信息，处理回滚操作（当一个查询或一组查询被中断时，数据库需要回到初始状态，一般用事务控制手段实现状态回滚），创建新的光标对象，等等
#而一个连接可以有很多个光标
#一个光标跟踪一种状态（state）信息，比如正在使用的是哪个数据库
#如果你有多个数据库，且需要向所有数据库写内容，就需要多个光标来进行处理
#光标还会包含最后一次查询执行的结果

In [40]:
#conn是连接对象
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='62581225liu', db='mysql', charset='utf8') 
#本示例代码中是使用 IP 连接的，即使用的是端口号（port）。因此，可将上诉代码改为：port=3306（注意：这里的3306为MySQL的默认端口号
#charset='utf8'要添加到连接字符串里。这是让连接 conn 把所有发送到数据库的信息都当成 UTF-8 编码格式（当然，前提是数据库默认编码已经设置成 UTF-8）

In [31]:
cur = conn.cursor() #cur是光标对象
cur.execute('USE scraping')

0

In [32]:
random.seed(datetime.datetime.now())

In [33]:
#把title, content两个参数添加到了一个 INSERT 语句中并用光标执行，然后用光标进行连接确认
#这是一个让光标与连接操作分离的好例子；当光标里存储了与数据库和数据库上下文（context）相关的信息时，它需要通过连接的确认操作先将信息传进数据库，再将信息插入数据库
def store(title, content):
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    cur.connection.commit()

In [34]:
def getLinks(articleUrl):
    html = urlopen('https://en.wikipedia.org' + articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

In [35]:
proxy = "https://10.204.32.132:7890" #代理服务器名：端口
proxy_py = urllib.request.ProxyHandler({'https':proxy}) #设置python认识的代理：默认为null
opener = urllib.request.build_opener(proxy_py) #建立
urllib.request.install_opener(opener) #运行代理

In [36]:
links = getLinks('/wiki/Kevin_Bacon')

In [37]:
#finally语句是在程序主循环的外面
#这样做可以保证，就算程序执行过程中发生中断或抛出异常（因为 Web 很复杂，所以你得随时准备遭遇异常），光标和连接都会在程序结束前立即关闭
try:
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links = getLinks(newArticle)
finally:
    cur.close() #记得把它们关闭，如果不关闭就会导致连接泄漏
    conn.close()

/wiki/8_(play)
/wiki/Ken_Leung
/wiki/Year_of_the_Fish
/wiki/Cinderella_effect
/wiki/Cinderella_Meets_Fella
/wiki/Tex_Avery_Screwball_Classics
/wiki/Aloha_Hooey
/wiki/Tortoise_Beats_Hare
/wiki/Cross-Country_Detours


KeyboardInterrupt: 